In [ ]:
from google.colab import drive

drive.mount('/content/drive')
root_dir = "/content/drive/My Drive/"
base_dir = root_dir + 'Tesi/'

In [ ]:
wiki_dir = root_dir + 'Tesi/Wiki50/Wikipedia/'

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
import nltk.tokenize.mwe
from nltk import tokenize
from nltk import pos_tag, word_tokenize

In [ ]:
!pip install stanza

In [ ]:
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline

In [ ]:
import re

In [ ]:
from itertools import chain
from collections import Counter

In [ ]:
from tqdm import tqdm
from time import sleep
tqdm.pandas()

DATASET

In [ ]:
import pandas as pd 

#df = pd.read_csv(base_dir + 'downstream/wikicorp_preprocessed.csv')

df = pd.read_csv(base_dir + 'downstream/ccoha2_preprocessed.csv') 

In [ ]:
df.shape

WIKIPEDIA DUMP

In [ ]:
with open(wiki_dir + 'wikipedia_list_preprocessed.txt') as f:
    wikipedia_list = f.read().splitlines() 

ENTITY EXTRACTION

In [ ]:
df["entities"]=""

In [ ]:
def entities_extraction(text):
    doc = nlp(text)

    entities= [ent.text for sentence in doc.sentences for ent in sentence.entities if ent.type in {"PERSON", "ORG", "GPE", "NORP", "FAC", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART", "LAW", "LANGUAGE", "MISC"}]

    return entities

In [ ]:
#df["entities"] = df['text'].apply(lambda x: entities_extraction(x))

df["entities"] = df['text'].progress_apply(lambda x: entities_extraction(x))

In [ ]:
entities_without_unigrams = []

In [ ]:
for i in range(len(df)):
  for kw in df['entities'][i]:
    if(' ' in kw):
        kw = kw.lower()
        entities_without_unigrams.append(kw)
  df['entities'][i]=entities_without_unigrams
  entities_without_unigrams = []

In [ ]:
entities_without_article = []

In [ ]:
for i in range(len(df)):
  for kw in df['entities'][i]:
    if('the' in kw[:4]):
        kw = re.sub("the ", "", kw)
        entities_without_article.append(kw)
    else:
        entities_without_article.append(kw)
  df['entities'][i]=entities_without_article
  entities_without_article = []

In [ ]:
for s in range(len(df)):  
  for i in range(len(df['entities'][s])):
    df['entities'][s][i] = df['entities'][s][i].split()

In [ ]:
for s in range(len(df)):  
  for i in range(len(df['entities'][s])-1,-1,-1):
    if (df['entities'][s][i][0] == df['entities'][s][i-1][-1]):
      df['entities'][s][i-1].remove(df['entities'][s][i-1][-1])
      df['entities'][s][i] = df['entities'][s][i-1] + df['entities'][s][i]
      df['entities'][s].remove(df['entities'][s][i-1])

In [ ]:
for s in range(len(df)):   
  for i in range(len(df['entities'][s])):
    df['entities'][s][i] = ' '.join(df['entities'][s][i])

In [ ]:
df

CON DATASET GIÀ MODIFICATO

In [ ]:
import pandas as pd 

df = pd.read_csv(base_dir + 'downstream/checkpoints/echeckpoint2_PoS-Ent.csv', converters={'entities': eval})

In [ ]:
df

POS

In [ ]:
df["pos"]=""

In [ ]:
def extract_phrases(my_tree, phrase):
   my_phrases = []
   if my_tree.label() == phrase:
      my_phrases.append(my_tree.copy(True))

   for child in my_tree:
       if type(child) is nltk.Tree:
            list_of_phrases = extract_phrases(child, phrase)
            if len(list_of_phrases) > 0:
                my_phrases.extend(list_of_phrases)

   return my_phrases

In [ ]:
grammar = "WP: {<NN><NN.?><NN.?><NN|NNS>|<NN|NNS><IN><DT>?<NN>|<NN><POS><NN|NNS>|<JJ><NN|NNS>+|<NN|NNS><IN><JJ><NN|NNS>|<IN><NN|NNS>|<NN|NNS><CC><NN|NNS>|<NN|NNS><NN>?<NN>?<NN|NNS>|<JJ>+<NN|NNS>}"

cp = nltk.RegexpParser(grammar)

In [ ]:
def pos_extraction(x):
    pos_list=[]

    sentence = pos_tag(tokenize.word_tokenize(x))
    tree = cp.parse(sentence)
    list_of_noun_phrases = extract_phrases(tree, 'WP')
    for phrase in list_of_noun_phrases:
      pos_list.append(" ".join([x[0] for x in phrase.leaves()]))

    text=pos_list
    
    return text

In [ ]:
df["pos"] = df['text'].progress_apply(lambda x: pos_extraction(x))

In [ ]:
pos_without_unigrams = []

In [ ]:
for i in range(len(df)):
  for kw in df['pos'][i]:
    if(' ' in kw):
        kw = kw.lower()
        pos_without_unigrams.append(kw)
  df['pos'][i]=pos_without_unigrams
  pos_without_unigrams = []

In [ ]:
for s in range(len(df)):  
  for i in range(len(df['pos'][s])):
    df['pos'][s][i] = df['pos'][s][i].split()

In [ ]:
for s in range(len(df)):  
  for i in range(len(df['pos'][s])-1,-1,-1):
    if (df['pos'][s][i][0] == df['pos'][s][i-1][-1]):
      df['pos'][s][i-1].remove(df['pos'][s][i-1][-1])
      df['pos'][s][i] = df['pos'][s][i-1] + df['pos'][s][i]
      df['pos'][s].remove(df['pos'][s][i-1])

In [ ]:
for s in range(len(df)):   
  for i in range(len(df['pos'][s])):
    df['pos'][s][i] = ' '.join(df['pos'][s][i])

In [ ]:
df

PREPROCESS

In [ ]:
''''
for i in range(len(df)):
  df['pos'][i] = list(dict.fromkeys(df['pos'][i]))
'''

In [ ]:
'''
for i in range(len(df)):
  df['entities'][i] = list(dict.fromkeys(df['entities'][i]))
'''

In [ ]:
df["united"] = df["pos"] + df["entities"]

In [ ]:
'''
for i in range(len(df)):
  df['united'][i] = list(dict.fromkeys(df['united'][i]))
'''

CON DATASET GIÀ MODIFICATO

In [ ]:
'''
import pandas as pd 
 
df = pd.read_csv(base_dir + 'checkpointText_PoS-Ent.csv', converters={'united': eval})
'''

In [ ]:
df

WIKIPEDIA COMPARISON

In [ ]:
df['check'] = ""

In [ ]:
wikipedia_list = set(wikipedia_list)

In [ ]:
df['check'] = [set(df['united'][i]).intersection(wikipedia_list) for i in tqdm(range(len(df)))]

MWE THRESHOLD

In [ ]:
mwe_counter = Counter(chain.from_iterable(df['check'])) 

In [ ]:
df['final'] = ""

In [ ]:
df['final'] = [([j for j in i if mwe_counter[j] >= 5]) for i in df['check']]

MWE SUBSTITUTION

In [ ]:
for i in range(len(df)):
  df['text'][i] = re.sub('('+'|'.join('\\b'+re.escape(g)+'\\b' for g in df['final'][i])+')', lambda m: m.group(0).replace(' ', '_'), df['text'][i], flags=re.I)

In [ ]:
df

In [ ]:
compression_opts = dict(method='zip', archive_name='newCCOHA2_PoS-Ent.csv')

df['text'].to_csv('newCCOHA2_PoS-Ent.zip', index=False, compression=compression_opts)